In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
dataset = pd.read_csv('/Users/aditya/Library/CloudStorage/OneDrive-Personal/AIR OneDrive/GenAI/ANN Churn Classification/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Dependent feature - Estimated Salary

# Preprocess the data
dataset = dataset.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Encode Categorical Variables
label_encoder_gender = LabelEncoder()
dataset['Gender'] = label_encoder_gender.fit_transform(dataset['Gender'])

dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
label_encoder_geography = OneHotEncoder()
geography_encoder = label_encoder_geography.fit_transform(dataset[['Geography']])
geography_encoded_df = pd.DataFrame(geography_encoder.toarray(), columns = label_encoder_geography.get_feature_names_out())


In [7]:
# Drop the Geography column from dataset
dataset = dataset.drop(['Geography'], axis = 1)

# Merge all columns
dataset = pd.concat([dataset, geography_encoded_df], axis = 1)

dataset.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Gender             10000 non-null  int64  
 2   Age                10000 non-null  int64  
 3   Tenure             10000 non-null  int64  
 4   Balance            10000 non-null  float64
 5   NumOfProducts      10000 non-null  int64  
 6   HasCrCard          10000 non-null  int64  
 7   IsActiveMember     10000 non-null  int64  
 8   EstimatedSalary    10000 non-null  float64
 9   Exited             10000 non-null  int64  
 10  Geography_France   10000 non-null  float64
 11  Geography_Germany  10000 non-null  float64
 12  Geography_Spain    10000 non-null  float64
dtypes: float64(5), int64(8)
memory usage: 1015.8 KB


In [9]:
# Splitting the dataset into features and target
X = dataset.drop('EstimatedSalary', axis = 1)
y = dataset['EstimatedSalary']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=7)

# Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [10]:
# Save the enbcoder and scaler for future use (Needed as this will deployed as streamlit web app)

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('label_encoder_geography.pkl', 'wb') as file:
    pickle.dump(label_encoder_geography,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [12]:
# Building ANN Model
model = Sequential ([
    Dense(64, activation = 'relu', input_shape=(X_train.shape[1],)), #Hidden Layer 1 with input layer - i/p layer only needed for hidden layer as its connnecting the input
    Dense(32, activation = 'relu'), # Hidden layer 2
    Dense(1) # Output layer - regression Activiation function will be used by default as it is a regression problem
]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# For performing forward and backward propagation, compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.mean_absolute_error

model.compile(optimizer = opt, loss = loss, metrics=['mae'])

In [14]:
# Setup Tensorboard - To Capture and visualise the logs
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #Creating the directory
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1) 

In [15]:
#Set EarlyStopping - To avoid unnecessary training of model(epochs) Will stop when there is less improvement in the model and loss is not reducing by much (based on set values)
early_stopping = EarlyStopping(monitor = 'val_loss', patience=10, restore_best_weights=True)

In [16]:
# Training the model
history = model.fit(
    X_train, 
    y_train, 
    validation_data = (X_test,y_test), 
    epochs=100,
    callbacks = [tensorflow_callback, early_stopping]
)

Epoch 1/100
250/250 [==============================] - 0s 668us/step - loss: 100060.2422 - mae: 100060.2422 - val_loss: 99820.6484 - val_mae: 99820.6484
Epoch 2/100
250/250 [==============================] - 0s 414us/step - loss: 99309.0781 - mae: 99309.0781 - val_loss: 98247.8516 - val_mae: 98247.8516
Epoch 3/100
250/250 [==============================] - 0s 416us/step - loss: 96547.6484 - mae: 96547.6484 - val_loss: 94103.9219 - val_mae: 94103.9219
Epoch 4/100
250/250 [==============================] - 0s 417us/step - loss: 91058.1953 - mae: 91058.1953 - val_loss: 87198.4062 - val_mae: 87198.4062
Epoch 5/100
250/250 [==============================] - 0s 411us/step - loss: 83105.0234 - mae: 83105.0234 - val_loss: 78242.4609 - val_mae: 78242.4609
Epoch 6/100
250/250 [==============================] - 0s 414us/step - loss: 73923.5469 - mae: 73923.5469 - val_loss: 69064.7969 - val_mae: 69064.7969
Epoch 7/100
250/250 [==============================] - 0s 407us/step - loss: 65128.9492 - ma

In [17]:
# Load tensorborad extension
%load_ext tensorboard

In [19]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6007 (pid 85716), started 0:00:12 ago. (Use '!kill 85716' to kill it.)

In [20]:
# Evaluating on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss} Test MAE: {test_mae}')

63/63 [==============================] - 0s 274us/step - loss: 49558.2539 - mae: 49558.2539
Test Loss: 49558.25390625 Test MAE: 49558.25390625


In [21]:
# Saving the model
model.save('regression_model.h5')

/Users/aditya/Library/CloudStorage/OneDrive-Personal/AIR OneDrive/GenAI/ANN Churn Classification/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
